In [ ]:
import json
import numpy as np
import pandas as pd
import pickle
from collections import Counter, defaultdict

In [2]:
classes_5 = ['Unrelated', 'Consistent', 'Indirect inconsistency', "Factual inconsistency", 'Surface contradiction']                             

### In this notebook, we analyze shifts participants made in their answers during repeated trial

In [3]:
with open(r'../data/qualtrics_survey/question_repeated_trial.pkl', 'rb') as f:
    question_repeated_trial = pickle.load(f)

In [ ]:
df = pd.read_csv("../data/df_gold.csv")

In [39]:
df

,Input,Answer_5_classes,Confidence_5_classes,All answers,Answer_3_classes,Explanations,LLaMA 8B 5 runs,LLaMA 70B 5 runs,Llama 8B majority 5 classes,Llama 70B majority 5 classes,...,ChatGPT-3.5 majority 5 classes,ChatGPT-4 majority 5 classes,LLaMA 8B majority 3 classes,LLaMA 70B majority 3 classes,ChatGPT-3.5 majority 3 classes,ChatGPT-4 majority 3 classes,ChatGPT-4 Confidence,ChatGPT-3.5 Confidence,LLaMA 8B Confidence,LLaMA 70B Confidence
0,Text 1: Motivated and confident workforce is a...,Indirect inconsistency,80.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,['Limiting employee influence goes against ide...,"['Indirect inconsistency', 'Factual inconsiste...","['Indirect inconsistency', 'Indirect inconsist...",Indirect inconsistency,Indirect inconsistency,...,Surface contradiction,Indirect inconsistency,Inconsistent,Inconsistent,Inconsistent,Inconsistent,100,80,80,100
1,Text 1: We want broad support for agriculture ...,Surface contradiction,60.000,"['Consistent', 'Surface contradiction', 'Surfa...",Inconsistent,['Broad support for all agriculture in incompa...,"['Factual inconsistency', 'Surface contradicti...","['Indirect inconsistency', 'Indirect inconsist...",Factual inconsistency,Indirect inconsistency,...,Factual inconsistency,Indirect inconsistency,Inconsistent,Inconsistent,Inconsistent,Inconsistent,100,60,80,100
2,Text 1: There should be a €29 ticket for every...,Surface contradiction,80.000,"['Indirect inconsistency', 'Surface contradict...",Inconsistent,0,"['Factual inconsistency', 'Indirect inconsiste...","['Indirect inconsistency', 'Indirect inconsist...",Factual inconsistency,Indirect inconsistency,...,Factual inconsistency,Indirect inconsistency,Inconsistent,Inconsistent,Inconsistent,Inconsistent,100,100,80,100
3,Text 1: We support the implementation of pilot...,Unrelated,60.000,"['Indirect inconsistency', 'Unrelated', 'Factu...",Unrelated,0,"['Unrelated', 'Consistent', 'Consistent', 'Con...","['Indirect inconsistency', 'Indirect inconsist...",Consistent,Indirect inconsistency,...,Indirect inconsistency,Unrelated,Consistent,Inconsistent,Inconsistent,Unrelated,100,80,80,100
4,Text 1: We fully support the conversion of add...,Indirect inconsistency,83.333,"['Unrelated', 'Indirect inconsistency', 'Indir...",Inconsistent,['Text A promotes cycling. Text B encourages c...,"['Unrelated', 'Consistent', 'Consistent', 'Con...","['Indirect inconsistency', 'Indirect inconsist...",Consistent,Indirect inconsistency,...,Factual inconsistency,Indirect inconsistency,Consistent,Inconsistent,Inconsistent,Inconsistent,100,80,80,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693,"Text 1: To bolster the economy, a major focus...",Indirect inconsistency,60.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,"[""Increasing interest rates may cause people t...","['Unrelated', 'Factual inconsistency', 'Indire...","['Indirect inconsistency', 'Factual inconsiste...",Indirect inconsistency,Factual inconsistency,...,Surface contradiction,Indirect inconsistency,Inconsistent,Inconsistent,Inconsistent,Inconsistent,80,60,60,80
694,Text 1: Students should have the free choice b...,Surface contradiction,60.000,"['Factual inconsistency', 'Factual inconsisten...",Inconsistent,0,"['Factual inconsistency', 'Surface contradicti...","['Indirect inconsistency', 'Indirect inconsist...",Surface contradiction,Indirect inconsistency,...,Surface contradiction,Indirect inconsistency,Inconsistent,Inconsistent,Inconsistent,Inconsistent,100,100,80,100
695,Text 1: Belonging to a certain social or ethni...,Indirect inconsistency,80.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,0,"['Indirect inconsistency', 'Indirect inconsist...","['Consistent', 'Consistent', 'Consistent', 'Co...",Consistent,Consistent,...,Surface contradiction,Consistent,Consistent,Consistent,Inconsistent,Consistent,100,80,60,100
696,Text 1: The level of social basic security mus...,Factual i

In [42]:
counter = 1
confidence_switched = {}
confidence_not_switched = {}

for class_name in classes_5:
    confidence_switched[class_name] = []
    confidence_not_switched[class_name] = []
    
for text in question_repeated_trial:
    print("New text")
    for annotator in question_repeated_trial[text]:
        print("Annotator", annotator)
        for class_name in classes_5:
            if question_repeated_trial[text][annotator][0] == class_name and question_repeated_trial[text][annotator][1] != class_name:
                print("Switch:", counter)
                print(text)
                counter += 1
                confidence_switched[class_name].append(df[df['Input']==text]['Confidence_5_classes'].iloc[0])
            # if question_repeated_trial[text][annotator][0] != question_repeated_trial[text][annotator][1] and question_repeated_trial[text][annotator][0] != class_name:
            #     print('someone else switched on that question too from', question_repeated_trial[text][annotator][0], 'to', question_repeated_trial[text][annotator][1])
            elif question_repeated_trial[text][annotator][0] == class_name and question_repeated_trial[text][annotator][1] == class_name:
                confidence_not_switched[class_name].append(df[df['Input']==text]['Confidence_5_classes'].iloc[0])

New text
Annotator v1
Annotator v2
New text
Annotator v1
Switch: 1
Text 1: We want broad support for agriculture with no limitations.

Text 2: We support the exclusive promotion of organic agriculture.
Annotator v2
Annotator v4
New text
Annotator v1
Switch: 2
Text 1: There should be a €29 ticket for everyone to use local public transport.

Text 2: Passengers need to pass a background-check and have stable income to qualify for the discounted ticket subscription.
Annotator v2
Annotator v7
New text
Annotator v9
New text
Annotator v1
Annotator v2
New text
Annotator v8
New text
Annotator v1
Annotator v2
New text
Annotator v1
Switch: 3
Text 1: Religious education in schools conveys regional culture.

Text 2: Confessional religious education must be abolished in state schools.
Annotator v2
Annotator v9
Switch: 4
Text 1: Religious education in schools conveys regional culture.

Text 2: Confessional religious education must be abolished in state schools.
New text
Annotator v1
Switch: 5
Text 1:

In [45]:
for class_name in confidence_not_switched:
    print(class_name)
    print("mean:", np.mean(confidence_not_switched[class_name]))
    print("mean:", np.mean(confidence_switched[class_name]))
    
    
    print("median:", np.median(confidence_not_switched[class_name]))
    print("median:", np.median(confidence_switched[class_name]))
    

Unrelated
mean: 71.53846153846153
mean: 53.333333333333336
median: 70.0
median: 40.0
Consistent
mean: 75.0
mean: 45.0
median: 80.0
median: 40.0
Indirect inconsistency
mean: 58.75
mean: 54.285714285714285
median: 60.0
median: 60.0
Factual inconsistency
mean: 50.588235294117645
mean: 58.8235294117647
median: 40.0
median: 60.0
Surface contradiction
mean: 64.21052631578948
mean: 67.14285714285714
median: 60.0
median: 60.0


In [17]:
change_per_class = dict()
# Only look at participants' second answer
change_per_class['2_times'] = dict()

for num_times in change_per_class:
    # for each number of switches, for each class create a dict
    for class_name in ['Unrelated', 'Consistent', 'Indirect inconsistency', 'Factual inconsistency', 'Surface contradiction']:
        change_per_class[num_times][class_name] = {}
        change_per_class[num_times][class_name]['answers'] = []
        change_per_class[num_times][class_name]['annotators'] = []
        
        
change_per_annotator = dict()
for i in range(1, 11, 1):
    change_per_annotator[f'v{i}'] = dict()
    for class_name in classes_5:
        change_per_annotator[f'v{i}'][class_name] = dict()
        change_per_annotator[f'v{i}']['total_repeated_questions'] = 0
        change_per_annotator[f'v{i}']['total_switches'] = 0
        change_per_annotator[f'v{i}'][class_name]['repeated_class'] = 0
        change_per_annotator[f'v{i}'][class_name]['switched'] = 0


In [19]:
# go through each sample which was repeatedly annotated
for el in question_repeated_trial:
    # look at each annotator
    for annotator in question_repeated_trial[el]:
        # their answers
        answers = question_repeated_trial[el][annotator]
        # this time we ignore how many times they switched, just look at the second answer.
    
        change_per_class['2_times'][answers[0]]['answers'].append(answers[1])
        change_per_class['2_times'][answers[0]]['annotators'].append(annotator)
        
        # we only add 1 becuase we consider only the first switch    
        change_per_annotator[annotator]['total_repeated_questions'] += 1
        change_per_annotator[annotator][answers[0]]['repeated_class'] += 1
        if answers[0] != answers[1]:
            change_per_annotator[annotator]['total_switches'] += 1
            change_per_annotator[annotator][answers[0]]['switched'] += 1
    

In [21]:
change_per_annotator

{'v1': {'Unrelated': {'repeated_class': 3, 'switched': 1},
  'total_repeated_questions': 18,
  'total_switches': 6,
  'Consistent': {'repeated_class': 4, 'switched': 1},
  'Indirect inconsistency': {'repeated_class': 6, 'switched': 2},
  'Factual inconsistency': {'repeated_class': 1, 'switched': 0},
  'Surface contradiction': {'repeated_class': 4, 'switched': 2}},
 'v2': {'Unrelated': {'repeated_class': 4, 'switched': 0},
  'total_repeated_questions': 18,
  'total_switches': 1,
  'Consistent': {'repeated_class': 2, 'switched': 0},
  'Indirect inconsistency': {'repeated_class': 5, 'switched': 0},
  'Factual inconsistency': {'repeated_class': 0, 'switched': 0},
  'Surface contradiction': {'repeated_class': 7, 'switched': 1}},
 'v3': {'Unrelated': {'repeated_class': 1, 'switched': 0},
  'total_repeated_questions': 30,
  'total_switches': 5,
  'Consistent': {'repeated_class': 2, 'switched': 0},
  'Indirect inconsistency': {'repeated_class': 3, 'switched': 1},
  'Factual inconsistency': {'r

now calculate percentage of times when people shifted per each class

In [26]:
for class_name in change_per_class['2_times']:
    print(class_name)
    answers = change_per_class['2_times'][class_name]['answers']
    annotators = change_per_class['2_times'][class_name]['annotators']

    switch_count = {}

    for annotator, answer in zip(annotators, answers):
        if answer != class_name:  # Check if it's a "switch"
            switch_count[annotator] = switch_count.get(annotator, 0) + 1

    sorted_switch_count = dict(sorted(switch_count.items(), key=lambda item: item[1], reverse=True))
    
    sorted_switch_percentage = dict()
    for annotator in sorted_switch_count:
        sorted_switch_percentage[annotator] = sorted_switch_count[annotator]/change_per_annotator[annotator][class_name]['repeated_class']
    # calculate percentage
    print(sorted_switch_count)
    print(sorted_switch_percentage)

Unrelated
{'v4': 3, 'v1': 1, 'v7': 1, 'v8': 1}
{'v4': 0.75, 'v1': 0.3333333333333333, 'v7': 0.5, 'v8': 0.16666666666666666}
Consistent
{'v4': 2, 'v1': 1, 'v9': 1}
{'v4': 0.2857142857142857, 'v1': 0.25, 'v9': 0.25}
Indirect inconsistency
{'v5': 4, 'v10': 4, 'v1': 2, 'v6': 1, 'v3': 1, 'v4': 1, 'v7': 1}
{'v5': 0.26666666666666666, 'v10': 0.5, 'v1': 0.3333333333333333, 'v6': 0.2, 'v3': 0.3333333333333333, 'v4': 0.5, 'v7': 0.08333333333333333}
Factual inconsistency
{'v10': 7, 'v3': 4, 'v7': 2, 'v4': 1, 'v6': 1, 'v9': 1, 'v8': 1}
{'v10': 0.6363636363636364, 'v3': 1.0, 'v7': 0.6666666666666666, 'v4': 0.3333333333333333, 'v6': 0.25, 'v9': 1.0, 'v8': 0.5}
Surface contradiction
{'v7': 4, 'v8': 3, 'v1': 2, 'v5': 2, 'v9': 1, 'v2': 1, 'v6': 1}
{'v7': 0.5714285714285714, 'v8': 1.0, 'v1': 0.5, 'v5': 0.4, 'v9': 0.16666666666666666, 'v2': 0.14285714285714285, 'v6': 0.2}


In [27]:
total_switches_per_class = dict()
for class_name in classes_5:
    total_switches_per_class[class_name] = dict()
    total_switches_per_class[class_name]['total_switch'] = 0
    total_switches_per_class[class_name]['total_repeated'] = 0

for class_name in classes_5:
    for annotator in change_per_annotator:
        total_switches_per_class[class_name]['total_switch'] += change_per_annotator[annotator][class_name]['switched']
        total_switches_per_class[class_name]['total_repeated'] += change_per_annotator[annotator][class_name]['repeated_class']

In [28]:
total_switches_per_class

{'Unrelated': {'total_switch': 6, 'total_repeated': 32},
 'Consistent': {'total_switch': 4, 'total_repeated': 36},
 'Indirect inconsistency': {'total_switch': 14, 'total_repeated': 70},
 'Factual inconsistency': {'total_switch': 17, 'total_repeated': 34},
 'Surface contradiction': {'total_switch': 14, 'total_repeated': 71}}

In [29]:
for class_name in total_switches_per_class:
    print(class_name, ":", total_switches_per_class[class_name]['total_switch']/total_switches_per_class[class_name]['total_repeated'])

Unrelated : 0.1875
Consistent : 0.1111111111111111
Indirect inconsistency : 0.2
Factual inconsistency : 0.5
Surface contradiction : 0.19718309859154928
